In [ ]:
import sys

import pprint as pp

sys.path.insert(0, "/home/peter/Documents/DataSet/")

import source.db as db

client = db.connect("localhost", 27017)

equivalent = client['refactoring']['equivalent']

count = equivalent.count_documents({})


In [ ]:
import difflib

file1_content = """This is the content of file 1.
It has some differences compared to file 2.
"""

file2_content = """This is the content of file 2.
It has some differences compared to file 1.
"""

# split the strings into lines
file1_lines = file1_content.splitlines()
file2_lines = file2_content.splitlines()

# use difflib to get the differences
differ = difflib.Differ()
diff = list(differ.compare(file1_lines, file2_lines))

# print the differences
for line in diff: print(line)


In [ ]:
record = {
  "_id": "21098ad7236a0e23761f71f2eb107d2ef68838a853dc7bc94379a10000d21dbd",
  "autopep": "import os\nimport sys\n\nfrom . import rsa_key_generator as rkg\n\nDEFAULT_BLOCK_SIZE = 128\nBYTE_SIZE = 256\n\n\ndef get_blocks_from_text(\n    message: str, block_size: int = DEFAULT_BLOCK_SIZE\n) -> list[int]:\n    message_bytes = message.encode(\"ascii\")\n\n    block_ints = []\n    for block_start in range(0, len(message_bytes), block_size):\n        block_int = 0\n        for i in range(block_start, min(block_start + block_size, len(message_bytes))):\n            block_int += message_bytes[i] * (BYTE_SIZE ** (i % block_size))\n        block_ints.append(block_int)\n    return block_ints\n\n\ndef get_text_from_blocks(\n    block_ints: list[int], message_length: int, block_size: int = DEFAULT_BLOCK_SIZE\n) -> str:\n    message: list[str] = []\n    for block_int in block_ints:\n        block_message: list[str] = []\n        for i in range(block_size - 1, -1, -1):\n            if len(message) + i < message_length:\n                ascii_number = block_int // (BYTE_SIZE**i)\n                block_int = block_int % (BYTE_SIZE**i)\n                block_message.insert(0, chr(ascii_number))\n        message.extend(block_message)\n    return \"\".join(message)\n\n\ndef encrypt_message(\n    message: str, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE\n) -> list[int]:\n    encrypted_blocks = []\n    n, e = key\n\n    for block in get_blocks_from_text(message, block_size):\n        encrypted_blocks.append(pow(block, e, n))\n    return encrypted_blocks\n\n\ndef decrypt_message(\n    encrypted_blocks: list[int],\n    message_length: int,\n    key: tuple[int, int],\n    block_size: int = DEFAULT_BLOCK_SIZE,\n) -> str:\n    decrypted_blocks = []\n    n, d = key\n    for block in encrypted_blocks:\n        decrypted_blocks.append(pow(block, d, n))\n    return get_text_from_blocks(decrypted_blocks, message_length, block_size)\n\n\ndef read_key_file(key_filename: str) -> tuple[int, int, int]:\n    with open(key_filename) as fo:\n        content = fo.read()\n    key_size, n, eor_d = content.split(\",\")\n    return (int(key_size), int(n), int(eor_d))\n\n\ndef encrypt_and_write_to_file(\n    message_filename: str,\n    key_filename: str,\n    message: str,\n    block_size: int = DEFAULT_BLOCK_SIZE,\n) -> str:\n    key_size, n, e = read_key_file(key_filename)\n    if key_size < block_size * 8:\n        sys.exit(\n            \"ERROR: Block size is {} bits and key size is {} bits. The RSA cipher \"\n            \"requires the block size to be equal to or greater than the key size. \"\n            \"Either decrease the block size or use different keys.\".format(\n                block_size * 8, key_size\n            )\n        )\n\n    encrypted_blocks = [str(i)\n                        for i in encrypt_message(message, (n, e), block_size)]\n\n    encrypted_content = \",\".join(encrypted_blocks)\n    encrypted_content = f\"{len(message)}_{block_size}_{encrypted_content}\"\n    with open(message_filename, \"w\") as fo:\n        fo.write(encrypted_content)\n    return encrypted_content\n\n\ndef read_from_file_and_decrypt(message_filename: str, key_filename: str) -> str:\n    key_size, n, d = read_key_file(key_filename)\n    with open(message_filename) as fo:\n        content = fo.read()\n    message_length_str, block_size_str, encrypted_message = content.split(\"_\")\n    message_length = int(message_length_str)\n    block_size = int(block_size_str)\n\n    if key_size < block_size * 8:\n        sys.exit(\n            \"ERROR: Block size is {} bits and key size is {} bits. The RSA cipher \"\n            \"requires the block size to be equal to or greater than the key size. \"\n            \"Did you specify the correct key file and encrypted file?\".format(\n                block_size * 8, key_size\n            )\n        )\n\n    encrypted_blocks = []\n    for block in encrypted_message.split(\",\"):\n        encrypted_blocks.append(int(block))\n\n    return decrypt_message(encrypted_blocks, message_length, (n, d), block_size)\n\n\ndef main() -> None:\n    filename = \"encrypted_file.txt\"\n    response = input(r\"Encrypt\\Decrypt [e\\d]: \")\n\n    if response.lower().startswith(\"e\"):\n        mode = \"encrypt\"\n    elif response.lower().startswith(\"d\"):\n        mode = \"decrypt\"\n\n    if mode == \"encrypt\":\n        if not os.path.exists(\"rsa_pubkey.txt\"):\n            rkg.make_key_files(\"rsa\", 1024)\n\n        message = input(\"\\nEnter message: \")\n        pubkey_filename = \"rsa_pubkey.txt\"\n        print(f\"Encrypting and writing to {filename}...\")\n        encrypted_text = encrypt_and_write_to_file(\n            filename, pubkey_filename, message)\n\n        print(\"\\nEncrypted text:\")\n        print(encrypted_text)\n\n    elif mode == \"decrypt\":\n        privkey_filename = \"rsa_privkey.txt\"\n        print(f\"Reading from {filename} and decrypting...\")\n        decrypted_text = read_from_file_and_decrypt(filename, privkey_filename)\n        print(\"writing decryption to rsa_decryption.txt...\")\n        with open(\"rsa_decryption.txt\", \"w\") as dec:\n            dec.write(decrypted_text)\n\n        print(\"\\nDecryption:\")\n        print(decrypted_text)\n\n\nif __name__ == \"__main__\":\n    main()\n",
  "autopep+for_to_comp": "import os\nimport sys\nfrom . import rsa_key_generator as rkg\nDEFAULT_BLOCK_SIZE = 128\nBYTE_SIZE = 256\n\n\ndef get_blocks_from_text(message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    message_bytes = message.encode('ascii')\n    block_ints = []\n    for block_start in range(0, len(message_bytes), block_size):\n        block_int = 0\n        for i in range(block_start, min(block_start + block_size, len(message_bytes))):\n            block_int += message_bytes[i] * BYTE_SIZE ** (i % block_size)\n        block_ints.append(block_int)\n    return block_ints\n\n\ndef get_text_from_blocks(block_ints: list[int], message_length: int, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    message: list[str] = []\n    for block_int in block_ints:\n        block_message: list[str] = []\n        for i in range(block_size - 1, -1, -1):\n            if len(message) + i < message_length:\n                ascii_number = block_int // BYTE_SIZE ** i\n                block_int = block_int % BYTE_SIZE ** i\n                block_message.insert(0, chr(ascii_number))\n        message.extend(block_message)\n    return ''.join(message)\n\n\ndef encrypt_message(message: str, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    encrypted_blocks = []\n    (n, e) = key\n    for block in get_blocks_from_text(message, block_size):\n        encrypted_blocks.append(pow(block, e, n))\n    return encrypted_blocks\n\n\ndef decrypt_message(encrypted_blocks: list[int], message_length: int, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    decrypted_blocks = []\n    (n, d) = key\n    for block in encrypted_blocks:\n        decrypted_blocks.append(pow(block, d, n))\n    return get_text_from_blocks(decrypted_blocks, message_length, block_size)\n\n\ndef read_key_file(key_filename: str) -> tuple[int, int, int]:\n    with open(key_filename) as fo:\n        content = fo.read()\n    (key_size, n, eor_d) = content.split(',')\n    return (int(key_size), int(n), int(eor_d))\n\n\ndef encrypt_and_write_to_file(message_filename: str, key_filename: str, message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    (key_size, n, e) = read_key_file(key_filename)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.'.format(block_size * 8, key_size))\n    encrypted_blocks = [str(i)\n                        for i in encrypt_message(message, (n, e), block_size)]\n    encrypted_content = ','.join(encrypted_blocks)\n    encrypted_content = f'{len(message)}_{block_size}_{encrypted_content}'\n    with open(message_filename, 'w') as fo:\n        fo.write(encrypted_content)\n    return encrypted_content\n\n\ndef read_from_file_and_decrypt(message_filename: str, key_filename: str) -> str:\n    (key_size, n, d) = read_key_file(key_filename)\n    with open(message_filename) as fo:\n        content = fo.read()\n    (message_length_str, block_size_str, encrypted_message) = content.split('_')\n    message_length = int(message_length_str)\n    block_size = int(block_size_str)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?'.format(block_size * 8, key_size))\n    encrypted_blocks = [int(block) for block in encrypted_message.split(',')]\n    return decrypt_message(encrypted_blocks, message_length, (n, d), block_size)\n\n\ndef main() -> None:\n    filename = 'encrypted_file.txt'\n    response = input('Encrypt\\\\Decrypt [e\\\\d]: ')\n    if response.lower().startswith('e'):\n        mode = 'encrypt'\n    elif response.lower().startswith('d'):\n        mode = 'decrypt'\n    if mode == 'encrypt':\n        if not os.path.exists('rsa_pubkey.txt'):\n            rkg.make_key_files('rsa', 1024)\n        message = input('\\nEnter message: ')\n        pubkey_filename = 'rsa_pubkey.txt'\n        print(f'Encrypting and writing to {filename}...')\n        encrypted_text = encrypt_and_write_to_file(\n            filename, pubkey_filename, message)\n        print('\\nEncrypted text:')\n        print(encrypted_text)\n    elif mode == 'decrypt':\n        privkey_filename = 'rsa_privkey.txt'\n        print(f'Reading from {filename} and decrypting...')\n        decrypted_text = read_from_file_and_decrypt(filename, privkey_filename)\n        print('writing decryption to rsa_decryption.txt...')\n        with open('rsa_decryption.txt', 'w') as dec:\n            dec.write(decrypted_text)\n        print('\\nDecryption:')\n        print(decrypted_text)\n\n\nif __name__ == '__main__':\n    main()\n",
  "autopep+ast_all": "import os\nimport sys\nfrom . import rsa_key_generator as rkg\nDEFAULT_BLOCK_SIZE = 128\nBYTE_SIZE = 256\n\n\ndef get_blocks_from_text(message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    message_bytes = message.encode('ascii')\n    block_ints = []\n    for block_start in range(0, len(message_bytes), block_size):\n        block_int = 0\n        for i in range(block_start, min(block_start + block_size, len(message_bytes))):\n            block_int += message_bytes[i] * BYTE_SIZE ** (i % block_size)\n        block_ints.append(block_int)\n    return block_ints\n\n\ndef get_text_from_blocks(block_ints: list[int], message_length: int, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    message: list[str] = []\n    for block_int in block_ints:\n        block_message: list[str] = []\n        for i in range(block_size - 1, -1, -1):\n            if len(message) + i < message_length:\n                ascii_number = block_int // BYTE_SIZE ** i\n                block_int = block_int % BYTE_SIZE ** i\n                block_message.insert(0, chr(ascii_number))\n        message.extend(block_message)\n    return ''.join(message)\n\n\ndef encrypt_message(message: str, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    encrypted_blocks = []\n    (n, e) = key\n    for block in get_blocks_from_text(message, block_size):\n        encrypted_blocks.append(pow(block, e, n))\n    return encrypted_blocks\n\n\ndef decrypt_message(encrypted_blocks: list[int], message_length: int, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    decrypted_blocks = []\n    (n, d) = key\n    for block in encrypted_blocks:\n        decrypted_blocks.append(pow(block, d, n))\n    return get_text_from_blocks(decrypted_blocks, message_length, block_size)\n\n\ndef read_key_file(key_filename: str) -> tuple[int, int, int]:\n    with open(key_filename) as fo:\n        content = fo.read()\n    (key_size, n, eor_d) = content.split(',')\n    return (int(key_size), int(n), int(eor_d))\n\n\ndef encrypt_and_write_to_file(message_filename: str, key_filename: str, message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    (key_size, n, e) = read_key_file(key_filename)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.'.format(block_size * 8, key_size))\n    encrypted_blocks = [str(i)\n                        for i in encrypt_message(message, (n, e), block_size)]\n    encrypted_content = ','.join(encrypted_blocks)\n    encrypted_content = f'{len(message)}_{block_size}_{encrypted_content}'\n    with open(message_filename, 'w') as fo:\n        fo.write(encrypted_content)\n    return encrypted_content\n\n\ndef read_from_file_and_decrypt(message_filename: str, key_filename: str) -> str:\n    (key_size, n, d) = read_key_file(key_filename)\n    with open(message_filename) as fo:\n        content = fo.read()\n    (message_length_str, block_size_str, encrypted_message) = content.split('_')\n    message_length = int(message_length_str)\n    block_size = int(block_size_str)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?'.format(block_size * 8, key_size))\n    encrypted_blocks = [int(block) for block in encrypted_message.split(',')]\n    return decrypt_message(encrypted_blocks, message_length, (n, d), block_size)\n\n\ndef main() -> None:\n    filename = 'encrypted_file.txt'\n    response = input('Encrypt\\\\Decrypt [e\\\\d]: ')\n    if not response.lower().startswith('e'):\n        if response.lower().startswith('d'):\n            mode = 'decrypt'\n    else:\n        mode = 'encrypt'\n    if not mode == 'encrypt':\n        if mode == 'decrypt':\n            privkey_filename = 'rsa_privkey.txt'\n            print(f'Reading from {filename} and decrypting...')\n            decrypted_text = read_from_file_and_decrypt(\n                filename, privkey_filename)\n            print('writing decryption to rsa_decryption.txt...')\n            with open('rsa_decryption.txt', 'w') as dec:\n                dec.write(decrypted_text)\n            print('\\nDecryption:')\n            print(decrypted_text)\n    else:\n        if not os.path.exists('rsa_pubkey.txt'):\n            rkg.make_key_files('rsa', 1024)\n        message = input('\\nEnter message: ')\n        pubkey_filename = 'rsa_pubkey.txt'\n        print(f'Encrypting and writing to {filename}...')\n        encrypted_text = encrypt_and_write_to_file(\n            filename, pubkey_filename, message)\n        print('\\nEncrypted text:')\n        print(encrypted_text)\n\n\nif __name__ == '__main__':\n    main()\n",
  "autopep+ast_all_but_invert_if": "import os\nimport sys\nfrom . import rsa_key_generator as rkg\nDEFAULT_BLOCK_SIZE = 128\nBYTE_SIZE = 256\n\n\ndef get_blocks_from_text(message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    message_bytes = message.encode('ascii')\n    block_ints = []\n    for block_start in range(0, len(message_bytes), block_size):\n        block_int = 0\n        for i in range(block_start, min(block_start + block_size, len(message_bytes))):\n            block_int += message_bytes[i] * BYTE_SIZE ** (i % block_size)\n        block_ints.append(block_int)\n    return block_ints\n\n\ndef get_text_from_blocks(block_ints: list[int], message_length: int, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    message: list[str] = []\n    for block_int in block_ints:\n        block_message: list[str] = []\n        for i in range(block_size - 1, -1, -1):\n            if len(message) + i < message_length:\n                ascii_number = block_int // BYTE_SIZE ** i\n                block_int = block_int % BYTE_SIZE ** i\n                block_message.insert(0, chr(ascii_number))\n        message.extend(block_message)\n    return ''.join(message)\n\n\ndef encrypt_message(message: str, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> list[int]:\n    encrypted_blocks = []\n    (n, e) = key\n    for block in get_blocks_from_text(message, block_size):\n        encrypted_blocks.append(pow(block, e, n))\n    return encrypted_blocks\n\n\ndef decrypt_message(encrypted_blocks: list[int], message_length: int, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    decrypted_blocks = []\n    (n, d) = key\n    for block in encrypted_blocks:\n        decrypted_blocks.append(pow(block, d, n))\n    return get_text_from_blocks(decrypted_blocks, message_length, block_size)\n\n\ndef read_key_file(key_filename: str) -> tuple[int, int, int]:\n    with open(key_filename) as fo:\n        content = fo.read()\n    (key_size, n, eor_d) = content.split(',')\n    return (int(key_size), int(n), int(eor_d))\n\n\ndef encrypt_and_write_to_file(message_filename: str, key_filename: str, message: str, block_size: int = DEFAULT_BLOCK_SIZE) -> str:\n    (key_size, n, e) = read_key_file(key_filename)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.'.format(block_size * 8, key_size))\n    encrypted_blocks = [str(i)\n                        for i in encrypt_message(message, (n, e), block_size)]\n    encrypted_content = ','.join(encrypted_blocks)\n    encrypted_content = f'{len(message)}_{block_size}_{encrypted_content}'\n    with open(message_filename, 'w') as fo:\n        fo.write(encrypted_content)\n    return encrypted_content\n\n\ndef read_from_file_and_decrypt(message_filename: str, key_filename: str) -> str:\n    (key_size, n, d) = read_key_file(key_filename)\n    with open(message_filename) as fo:\n        content = fo.read()\n    (message_length_str, block_size_str, encrypted_message) = content.split('_')\n    message_length = int(message_length_str)\n    block_size = int(block_size_str)\n    if key_size < block_size * 8:\n        sys.exit('ERROR: Block size is {} bits and key size is {} bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?'.format(block_size * 8, key_size))\n    encrypted_blocks = [int(block) for block in encrypted_message.split(',')]\n    return decrypt_message(encrypted_blocks, message_length, (n, d), block_size)\n\n\ndef main() -> None:\n    filename = 'encrypted_file.txt'\n    response = input('Encrypt\\\\Decrypt [e\\\\d]: ')\n    if response.lower().startswith('e'):\n        mode = 'encrypt'\n    elif response.lower().startswith('d'):\n        mode = 'decrypt'\n    if mode == 'encrypt':\n        if not os.path.exists('rsa_pubkey.txt'):\n            rkg.make_key_files('rsa', 1024)\n        message = input('\\nEnter message: ')\n        pubkey_filename = 'rsa_pubkey.txt'\n        print(f'Encrypting and writing to {filename}...')\n        encrypted_text = encrypt_and_write_to_file(\n            filename, pubkey_filename, message)\n        print('\\nEncrypted text:')\n        print(encrypted_text)\n    elif mode == 'decrypt':\n        privkey_filename = 'rsa_privkey.txt'\n        print(f'Reading from {filename} and decrypting...')\n        decrypted_text = read_from_file_and_decrypt(filename, privkey_filename)\n        print('writing decryption to rsa_decryption.txt...')\n        with open('rsa_decryption.txt', 'w') as dec:\n            dec.write(decrypted_text)\n        print('\\nDecryption:')\n        print(decrypted_text)\n\n\nif __name__ == '__main__':\n    main()\n",
  "_source": "import os\nimport sys\n\nfrom . import rsa_key_generator as rkg\n\nDEFAULT_BLOCK_SIZE = 128\nBYTE_SIZE = 256\n\n\ndef get_blocks_from_text(\n    message: str, block_size: int = DEFAULT_BLOCK_SIZE\n) -> list[int]:\n    message_bytes = message.encode(\"ascii\")\n\n    block_ints = []\n    for block_start in range(0, len(message_bytes), block_size):\n        block_int = 0\n        for i in range(block_start, min(block_start + block_size, len(message_bytes))):\n            block_int += message_bytes[i] * (BYTE_SIZE ** (i % block_size))\n        block_ints.append(block_int)\n    return block_ints\n\n\ndef get_text_from_blocks(\n    block_ints: list[int], message_length: int, block_size: int = DEFAULT_BLOCK_SIZE\n) -> str:\n    message: list[str] = []\n    for block_int in block_ints:\n        block_message: list[str] = []\n        for i in range(block_size - 1, -1, -1):\n            if len(message) + i < message_length:\n                ascii_number = block_int // (BYTE_SIZE**i)\n                block_int = block_int % (BYTE_SIZE**i)\n                block_message.insert(0, chr(ascii_number))\n        message.extend(block_message)\n    return \"\".join(message)\n\n\ndef encrypt_message(\n    message: str, key: tuple[int, int], block_size: int = DEFAULT_BLOCK_SIZE\n) -> list[int]:\n    encrypted_blocks = []\n    n, e = key\n\n    for block in get_blocks_from_text(message, block_size):\n        encrypted_blocks.append(pow(block, e, n))\n    return encrypted_blocks\n\n\ndef decrypt_message(\n    encrypted_blocks: list[int],\n    message_length: int,\n    key: tuple[int, int],\n    block_size: int = DEFAULT_BLOCK_SIZE,\n) -> str:\n    decrypted_blocks = []\n    n, d = key\n    for block in encrypted_blocks:\n        decrypted_blocks.append(pow(block, d, n))\n    return get_text_from_blocks(decrypted_blocks, message_length, block_size)\n\n\ndef read_key_file(key_filename: str) -> tuple[int, int, int]:\n    with open(key_filename) as fo:\n        content = fo.read()\n    key_size, n, eor_d = content.split(\",\")\n    return (int(key_size), int(n), int(eor_d))\n\n\ndef encrypt_and_write_to_file(\n    message_filename: str,\n    key_filename: str,\n    message: str,\n    block_size: int = DEFAULT_BLOCK_SIZE,\n) -> str:\n    key_size, n, e = read_key_file(key_filename)\n    if key_size < block_size * 8:\n        sys.exit(\n            \"ERROR: Block size is {} bits and key size is {} bits. The RSA cipher \"\n            \"requires the block size to be equal to or greater than the key size. \"\n            \"Either decrease the block size or use different keys.\".format(\n                block_size * 8, key_size\n            )\n        )\n\n    encrypted_blocks = [str(i) for i in encrypt_message(message, (n, e), block_size)]\n\n    encrypted_content = \",\".join(encrypted_blocks)\n    encrypted_content = f\"{len(message)}_{block_size}_{encrypted_content}\"\n    with open(message_filename, \"w\") as fo:\n        fo.write(encrypted_content)\n    return encrypted_content\n\n\ndef read_from_file_and_decrypt(message_filename: str, key_filename: str) -> str:\n    key_size, n, d = read_key_file(key_filename)\n    with open(message_filename) as fo:\n        content = fo.read()\n    message_length_str, block_size_str, encrypted_message = content.split(\"_\")\n    message_length = int(message_length_str)\n    block_size = int(block_size_str)\n\n    if key_size < block_size * 8:\n        sys.exit(\n            \"ERROR: Block size is {} bits and key size is {} bits. The RSA cipher \"\n            \"requires the block size to be equal to or greater than the key size. \"\n            \"Did you specify the correct key file and encrypted file?\".format(\n                block_size * 8, key_size\n            )\n        )\n\n    encrypted_blocks = []\n    for block in encrypted_message.split(\",\"):\n        encrypted_blocks.append(int(block))\n\n    return decrypt_message(encrypted_blocks, message_length, (n, d), block_size)\n\n\ndef main() -> None:\n    filename = \"encrypted_file.txt\"\n    response = input(r\"Encrypt\\Decrypt [e\\d]: \")\n\n    if response.lower().startswith(\"e\"):\n        mode = \"encrypt\"\n    elif response.lower().startswith(\"d\"):\n        mode = \"decrypt\"\n\n    if mode == \"encrypt\":\n        if not os.path.exists(\"rsa_pubkey.txt\"):\n            rkg.make_key_files(\"rsa\", 1024)\n\n        message = input(\"\\nEnter message: \")\n        pubkey_filename = \"rsa_pubkey.txt\"\n        print(f\"Encrypting and writing to {filename}...\")\n        encrypted_text = encrypt_and_write_to_file(filename, pubkey_filename, message)\n\n        print(\"\\nEncrypted text:\")\n        print(encrypted_text)\n\n    elif mode == \"decrypt\":\n        privkey_filename = \"rsa_privkey.txt\"\n        print(f\"Reading from {filename} and decrypting...\")\n        decrypted_text = read_from_file_and_decrypt(filename, privkey_filename)\n        print(\"writing decryption to rsa_decryption.txt...\")\n        with open(\"rsa_decryption.txt\", \"w\") as dec:\n            dec.write(decrypted_text)\n\n        print(\"\\nDecryption:\")\n        print(decrypted_text)\n\n\nif __name__ == \"__main__\":\n    main()\n"
}

print(record["autopep+for_to_comp"])